In [6]:
_PROMPT_TEMPLATE = """
请你提取下面段落的关键词，要求所提取的关键词能够鲜明表示内容性质。所提取的关键词应不少于5个。

待处理内容：

本次股票发行后拟在创业板市场上市，该市场具有较高的投资风险。创业
板公司具有业绩不稳定、经营风险高、退市风险大等特点，投资者面临较大的
市场风险。投资者应充分了解创业板市场的投资风险及本公司所披露的风险因
素，审慎作出投资决定。
华瑞电器股份有限公司
Huarui Electrical Appliance Co., Ltd.
册地址：宁波市鄞州区姜山镇科技园区
首次公开发行股票并在创业板上市
招股意向书
发行人及全体董事、监事、高级管理人员承诺招股意向书不存在虚假记载、
误导性陈述或重大遗漏，并对其真实性、准确性、完整性、及时性承担个别和连
带的法律责任。
发行人及全体董事、监事、高级管理人员、发行人的控股股东、实际控制人
以及保荐人、承销的证券公司承诺因发行人招股意向书及其他信息披露资料有虚
假记载、误导性陈述或者重大遗漏，致使投资者在证券发行和交易中遭受损失的，
将依法赔偿投资者损失。

注意：你的回答不需要分段。你的回答格式应该按照下面的内容，请注意---output 等标记都必须输出，这是我用来提取答案的标记。
除了你的回答之外不要输出多余的文字。

---output
${{"keywords":[你提取的关键词]}}
"""

In [1]:
import dashscope
from dashscope import Generation
from http import HTTPStatus
import random
from CustomLLM import QWENonlie
from Container import g_container
apikey='sk-02b8b1a9175a4cae9bf9571d517d8b81'
llm=QWENonlie.from_name(model_name=Generation.Models.qwen_max,api_key=apikey)
g_container.MODEL = llm

In [9]:
llm._call(prompt=_PROMPT_TEMPLATE,
          stop=['---stop'])

'---output\n{"keywords":["创业板","投资风险","市场风险","信息披露","法律责任"]}'

In [2]:
%load_ext autoreload
%autoreload 2
from config import QWEN_CONFIG
from dashscope import TextEmbedding,BatchTextEmbedding
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import DashScopeEmbeddings,HuggingFaceEmbeddings

emb2=HuggingFaceEmbeddings(model_name='../../model/m3e/',encode_kwargs={'normalize_embeddings':True})
g_container.EMBEDDING=emb2


In [3]:
%load_ext autoreload
%autoreload 2
from config import QWEN_CONFIG
cfg = QWEN_CONFIG(api_key=apikey,
                  embedding_name='text_embedding_async_v1')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
%load_ext autoreload
%autoreload 2
from TxtProcess import load_dir_txt_idx_process
cfg = QWEN_CONFIG(api_key='apikey',
                  embedding_name='text_embedding_async_v1')
kb=load_dir_txt_idx_process(txt_dir='../../bs_challenge_financial_14b_dataset/pdf_txt_file/',
                     pdf_dir='../../bs_challenge_financial_14b_dataset/pdf/',
                     cfg=cfg
                     )


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


handle files::   0%|          | 0/80 [00:00<?, ?it/s]

handle tables::   0%|          | 0/386 [00:00<?, ?it/s]

handle tables::   0%|          | 0/326 [00:00<?, ?it/s]

handle tables::   0%|          | 0/713 [00:00<?, ?it/s]

...

handle tables::   0%|          | 0/489 [00:00<?, ?it/s]

handle tables::   0%|          | 0/289 [00:00<?, ?it/s]

handle tables::   0%|          | 0/1191 [00:00<?, ?it/s]

handle tables::   0%|          | 0/317 [00:00<?, ?it/s]

+----------------------------------------------+--------------------------------------------------------------------------------------------+---------+------------+-------------------------------------------------------------------------------------------------------------------+------------------------------------------+
|                    source                    |                                          company                                           | doc_len | chunk_size |                                                     meta_data                                                     |                save_index                |
+----------------------------------------------+--------------------------------------------------------------------------------------------+---------+------------+-------------------------------------------------------------------------------------------------------------------+------------------------------------------+
| 03c625c108ac0137f413dfd413

In [60]:
from TxtProcess import faiss_kb_ds
kbidx=faiss_kb_ds(vs_path=cfg.vec_store_path,
                  kb_path='./',
                  embs=g_container.EMBEDDING)
kbidx.do_load(index='index')
kbidx.do_view()

32

In [4]:
%load_ext autoreload
%autoreload 2
from Tools.konwlegeqa import run_knowledge_qa

run_knowledge_qa(query='广州中海达卫星导航技术股份有限公司的子公司中海达测绘的主营业务是什么？',
                 cfg=cfg)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


{'inputs': '广州中海达卫星导航技术股份有限公司的子公司中海达测绘的主营业务是什么？',
 'outputs': '中海达测绘的主营业务是卫星导航技术。',
 'relent_docs': ['91b4426b075560a1a45247f9cfa9fa73d56c945c']}

In [11]:
import pdfplumber 
from prettytable import PrettyTable
43571629ce9fb30b44ccefdca9f637102c16d1a2.txt
with pdfplumber.open('../../bs_challenge_financial_14b_dataset/pdf/ba561001c2abd0714c413e2f6a77ffee58c644a6.PDF') as f:
    tables=f.pages[51]
type(tables)

pdfplumber.page.Page

In [44]:
from TxtProcess import  faiss_kb_ds
tttt=faiss_kb_ds(cfg.vec_store_path,kb_path='./',embs=emb2)
tttt.do_load(index='a0d278016e9baded15e0f5b3964563e525b1b787')

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader,PDFPlumberLoader

tt=PDFPlumberLoader(file_path='../../bs_challenge_financial_14b_dataset/pdf/a244b6ed9da7411f804e62b82a8fdfc015dff284.PDF').load()

In [1]:
import sqlite3 
conn=sqlite3.connect('../../bs_challenge_financial_14b_dataset/dataset/博金杯比赛数据.db')
cursor=conn.cursor()

In [8]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# 获取查询结果
tables = cursor.fetchall()
# 遍历所有表
res={
    'db_id': 'department_management',
    'db_path': './spider/database',
    'db_table_names':[],
    'db_column_names':[],
    'db_primary_keys':[]
}
for i,table in enumerate(tables):
    table_name = table[0]
    res['db_table_names'].append({
        'table_id':i,
        'table_name':table_name})
    # 查询表结构
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    for column in columns:
        res['db_column_names'].append({
                'table_id':i,
                'column_name':column[1],
                'column_type':column[2]
            })
        if column[5] == 1:
            res['db_primary_keys'].append({
                'table_id':i,
                'column_name':column[1]
            })
res

{'db_id': 'department_management',
 'db_path': './spider/database',
 'db_table_names': [{'table_id': 0, 'table_name': '基金基本信息'},
  {'table_id': 1, 'table_name': '基金股票持仓明细'},
  {'table_id': 2, 'table_name': '基金债券持仓明细'},
  {'table_id': 3, 'table_name': '基金可转债持仓明细'},
  {'table_id': 4, 'table_name': '基金日行情表'},
  {'table_id': 5, 'table_name': 'A股票日行情表'},
  {'table_id': 6, 'table_name': '港股票日行情表'},
  {'table_id': 7, 'table_name': 'A股公司行业划分表'},
  {'table_id': 8, 'table_name': '基金规模变动表'},
  {'table_id': 9, 'table_name': '基金份额持有人结构'}],
 'db_column_names': [{'table_id': 0,
   'column_name': '基金代码',
   'column_type': 'TEXT'},
  {'table_id': 0, 'column_name': '基金全称', 'column_type': 'TEXT'},
  {'table_id': 0, 'column_name': '基金简称', 'column_type': 'TEXT'},
  {'table_id': 0, 'column_name': '管理人', 'column_type': 'TEXT'},
  {'table_id': 0, 'column_name': '托管人', 'column_type': 'TEXT'},
  {'table_id': 0, 'column_name': '基金类型', 'column_type': 'TEXT'},
  {'table_id': 0, 'column_name': '成立日期', 'column_type'